##### 如何生成序列数据
将循环神经网络用于生成序列数据. 使用前面的标记作为输入, 训练一个网络来预测序列中接下来的一个或多个标记. 标记通常是单词或字符, 给定前面的标记, 能够对下一个标记的概率进行建模的任何网络都叫做语言模型. 语言模型能够捕捉到语言的潜在空间, 即语言的统计结构.
一旦训练好了这样一个语言模型, 就可以从中采样(sample, 即生成新序列). 向模型中输入一个初始文本字符串[即条件数据], 要求模型生成下一个字符或者下一个单词, 然后将生成的输出添加到输入数据中去. 多次重复这一过程, 这个循环可以生成任意长度的序列, 该序列反映了模型训练数据的结构.
##### 采样策略的重要性
生成文本时,如何选择下一个字符至关重要, 一种简单的方法叫贪婪采样(greedy sampling), 即始终选择概率最大的字符, 这种方法会得到重复的, 可预测的字符; 另一种方法是在采样的过程中引入随机性, 从下一个字符的概率分布中进行采样, 这叫做随机采样(stochastic samplit). 随机采样虽然能够表现出创造性, 但是在采样的过程中无法控制随机性的大小.  
为了控制采样过程中随机性的大小, 我们引入了一个叫作softmax温度的参数, 用于表示采样分布的熵, 即表示所选的下一个字符会有多么的出人意料或者多么的可预测. 给定一个temperature值,对softmax的输出进行重新加权, 计算得到一个新的概率分布.

In [ ]:
import numpy as np
# original_distriburion: 概率值组成的一维Numpy数组, 概率值之和必须等于1
# temperature: 用于定量描述输出分布的熵
def reweight_distribution(original_distriburion, temperature=0.5):
    distribution = np.log(original_distriburion) / temperature
    distribution = np.exp(distribution)
    # 返回原始分布重新加权后的结果, distribution的求和肯呢个不再等于1
    # 因此, 将它除以求和一得到新的分布
    return distribution / np.sum(distribution)

更高的温度得到的是熵更大的采样分布, 会生成更加出人意料的, 更加无结构的生成数据; 更小的温度值对应更小的随机性, 以及更加可预测的生成数据.

#### 字符级的LSTM文本生成
针对尼采的写作风格和主题的模型

In [ ]:
# Get data
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

In [ ]:
# Extract data
# 提取60个字符组成的序列
maxlen = 60
# 每三个字符采样一个新序列
step = 3
# 保存所提取的序列
sentences = []
# 保存目标(即下一个字符)
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text))) # list of unique chars in corpurs
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
# Build network
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

目标是one-hot编码, 所以训练模型需要使用categorical_crossentropy作为损失.

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

#### 训练语言模型并从中采样
给定一个训练好的模型和一个种子文本片段, 我们可以通过重复以下操作来生成新的文本:
- 给定目前已生成的文本, 从模型中得到下一个字符的概率分布.
- 根据某个温度对分布进行重新加权
- 根据重新加权的分布对下一个字符进行随机采样
- 将新字符添加到文本末尾

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

文本生成循环, 反复训练并生成文本, 每轮过后都使用一系列不同的温度值来生成文本.

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices]

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

较小的温度值会得到极端重复和可预测的文本, 但局部结构是非常真实的, 特别是所有的单词都是真实的英文单词. 随着温度值越来越大, 生成的文本更加出人意料, 更加具有创造性, 会出现全新的单词. 对于较大的温度值, 大部分的单词看起来像是半随机的字符串.

小结:
- 我们可以生成离散的序列数据: 给定前面的标记, 训练模型来预测接下来的一个或多个标记.
- 对于文本来说, 这种模型叫做语言模型. 它可以是单词级的,也可以是字符级的
- 对下一个标记进行采样, 需要在坚持模型的判断与引入随机性之间寻找平衡.
- 使用softmax温度, 一定要尝试多种不同的温度, 以找到合适的一个

TODO: DeepDream